Run this **once** to add `sorcha` to our path

In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import sorcha
import subprocess
from multiprocessing import Pool
os.environ["PATH"] = "/epyc/users/mihael/vsa/sorcha-env/bin:" + os.environ["PATH"]

BASE_DIR = "./real_data/"
file_prefix = ""

ModuleNotFoundError: No module named 'sorcha'

In [78]:


def splitfile(input_file, output_dir, n):
    
"""
input_file --> file you want to split
output_dir --> where you want to put the chunks that the file will be split into
n --> number of the chunks 
"""

# === Step 1: Read all lines from the file ===
    with open(input_file, 'r', encoding='utf-8') as f:
        lines = [line.rstrip('\n') for line in f if line.strip()]

# === Step 2: Extract and clean header ===
    raw_header = lines[0]
    header_fields = [h.strip() for h in raw_header.split(',') if h.strip()]
    num_columns = len(header_fields)

# === Step 3: Clean and normalize data rows ===
    data = []
    for line_num, line in enumerate(lines[1:], start=2):
        fields = [f.strip() for f in line.split(',')]
    
    # Remove completely empty strings created by double tabs
        fields = [f for f in fields if f != '']

        if len(fields) < num_columns:
            fields += [''] * (num_columns - len(fields))  # pad missing
        elif len(fields) > num_columns:
            fields = fields[:num_columns]  # trim extras

        if len(fields) != num_columns:
            print(f"⚠️  Line {line_num} still malformed: {len(fields)} fields (expected {num_columns})")
    
        data.append(fields)

# === Step 4: Create DataFrame ===
    df = pd.DataFrame(data, columns=header_fields)

# === Step 5: Split and export ===
    chunks = np.array_split(df, n)
    os.makedirs(output_dir, exist_ok=True)

    for i, chunk in enumerate(tqdm(chunks, desc="Writing files")):
        out_path = os.path.join(output_dir, f'{input_file.split('/')[-1]}_split_{i+1}.csv')
        chunk.to_csv(out_path, sep=',', index=False)

    print(f"✅ Finished splitting. {n} clean files saved in '{output_dir}'")
splitfile(f"{BASE_DIR}/{file_prefix}colors.csv",f"{BASE_DIR}/{file_prefix}colours", 150)
splitfile(f"{BASE_DIR}/{file_prefix}orbits.csv",f"{BASE_DIR}/{file_prefix}orbits", 150)

Writing files: 100%|██████████| 150/150 [00:03<00:00, 39.52it/s]


✅ Finished splitting. 150 clean files saved in './real_data//colours'


Writing files: 100%|██████████| 150/150 [00:04<00:00, 34.66it/s]


✅ Finished splitting. 150 clean files saved in './real_data//orbits'


In [2]:
#run sorcha for the chosen file
def run_sorcha(i):
    subprocess.run(f"sorcha run -c ./sorcha_config.ini -p {BASE_DIR}/{file_prefix}colours/{file_prefix}colors.csv_split_{i}.csv  --orbits {BASE_DIR}/{file_prefix}orbits/orbits.csv_split_{i}.csv --pointing-db sv_20250729.db -o {BASE_DIR}/output/ -t e2e/e2e_{i} --stats stats/stats_{i} -f", shell=True)
    

In [84]:
run_sorcha(150)

/astro/store/epyc/users/mihael/vsa/sorcha-env/lib/python3.12/site-packages/assist/__init__.py:44: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


KeyboardInterrupt: 

In [82]:
#run the sorcha on the chunks simultaneously
indices = range(1, 151)
with Pool() as pool:
    for _ in tqdm(pool.imap_unordered(run_sorcha, indices), tot_orbitsal=len(indices)):
        pass
subprocess.run(f"rm {BASE_DIR}/output/*.log & rm {BASE_DIR}/output/*.err", shell=True)



  0%|          | 0/150 [00:00<?, ?it/s]/astro/store/epyc/users/mihael/vsa/sorcha-env/lib/python3.12/site-packages/assist/__init__.py:44: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
/astro/store/epyc/users/mihael/vsa/sorcha-env/lib/python3.12/site-packages/assist/__init__.py:44: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
/astro/store/epyc/users/mihael/vsa/sorcha-env/lib/python3.12/site-packages/assist/__init__.py:44: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources packag

CompletedProcess(args='rm ./real_data//output/*.log & rm ./real_data//output/*.err & mv ./real_data//output/e2e*.csv ./real_data//output/e2e/ & mv ./real_data//output/stats*.csv ./real_data//output/stats/', returncode=0)

In [69]:
#merge the files so we don't have to open each chunk individually
def merge_csv_files(folder_path, output_file):
    """
    folder_path -> path to the source folder
    output_file --> file made of merged chunks
    
    """
    all_files = sorted([
        os.path.join(folder_path, f)
        for f in os.listdir(folder_path)
        if f.endswith('.csv')
    ])
    df_list = [pd.read_csv(file) for file in all_files]
    merged_df = pd.concat(df_list, ignore_index = True)
    merged_df.to_csv(output_file, index= False)
    print(f'Merged {len(all_files)} files into: {output_file}')


In [83]:
merge_csv_files(f"{BASE_DIR}/output/e2e/",f"{BASE_DIR}/{file_prefix}e2e.csv") 
merge_csv_files(f"{BASE_DIR}output/stats/",f"{BASE_DIR}/{file_prefix}stats.csv")

Merged 150 files into: ./real_data//e2e.csv
Merged 150 files into: ./real_data//stats.csv


In [1]:
! pwd

/astro/store/epyc/users/mihael/vsa
